## 학습정리

### 25장. 맵리듀스
* 맵리듀스(MapReduce) : 빅데이터를 병렬 처리할 때 사용하는 프로그래밍 모델
* 맵리듀스 알고리즘의 원리 
    1. mapper(map)함수로 각 아이템을 0개 이상의 키-값 쌍으로 변환 (파이썬의 map함수와는 다른 함수)
        * mapper : 키,값으로 변환 
    2. 동일한 키를 가진 모든 키-값 쌍을 모음
    3. reducer 함수로 같은 키를 가진 쌍들에 대해 원하는 방식으로 처리된 값을 반환 
        * reucer : 원하는 방식으로 처리된 값을 반환 
####  25.1 예시 : 단어 수 세기
* 게시물에 등장하는 단어들의 빈도수를 세서 가장 높은 단어들을 찾기
    * 데이터의 양이 너무 많아서 컴퓨터 한 대로는 부족할 정도가 된다면 : 맵리듀스
       * 빅데이터 인프라를 사용가능 
* 맵리듀스 모델 
    * 키-값 쌍으로 변환시켜 주는 함수
        * 키 : 단어
    * 최종 결과물 : 단어별 빈도수
        
####  25.2 왜 맵리듀스인가?
* 맵리듀스의 가장 큰 장점 : 데이터 처리(함수)를 데이터로 가져가서 함
    * 기존의 방식 : 모든 문서가 한 컴퓨터의 저장 공간 안에 있거나 프로세싱 도중 그 컴퓨터에 전달되어야 함
    * 제대로된 인프라가 같여있을 때 맵리듀스 방식
        * 각 컴퓨터는 자신이 보유하고 있는 문서에 mapper를 수행하고, 여러 개의 키-값 쌍을 생성
        * 키-값 쌍들을 지정된 몇 개의 리듀스 작업을 수행하는 컴퓨터로 보냄
        * 리듀스 작업을 수행하는 각 컴퓨터는 키별로 쌍들을 묶고 reducer를 실행
        * 각 키-최종 값 쌍을 반환
    * 수평적 확장이 용이 : 컴퓨터의 개수를 두 배로 늘리면 계산도 두 배 빨라짐
    
####  25.3 맵리듀스 일반화하기
* 단어수 세기 함수를 더 일반화 시켜서 프레임워크를 얻을 수 있음

####  25.4 예시 : 사용자의 글 분석하기
* 주중 어떤 요일에 사람들이 데이터 과학에 대해 가장 많이 얘기를 나누는지 
* 각 사용자가 어떤 단어를 가장 많이 사용했는지 분석
    * mapper 작성 방법
        * 가장 좋은 방법 : 사용자의 이름을 키로 사용, 단어와 단어의 수를 값으로 사용
        * 단어를 키로 사용, 사용자의 이름과 단어의 수를 값으로 사용
        * 사용자의 이름과 단어를 키로 사용하고, 단어의 수를 값으로 사용 
    
####  25.5 행렬 곱셈
* 행렬 곱셈을 리스트로 표현 
* 행렬이 sparse할 때는 리스트의 리스트 형태는 비효율적 : 0이 아닌 값을 담고 있는 위치만 효율적으로 저장 

####  25.6 Combiner
* 분산적인 환경에서는 컴퓨터와 컴퓨터 사이에서 전달되어야 하는 데이터의 양을 줄이기 위해 combiner를 사용할 수 있음
    * 컴퓨터 간 데이터 전송량을 줄여줌, 속도 증가 

####  25.7 더 공부해 보고 싶다면
* 맵리듀스는 인기가 많이 사그라듬. 시간을 많이 투자할 가치는 별로 없음
* 하둡(Hadoop)은 가장 널리 사용되는 맵리듀스 시스템
* 아마존 AWS에서는 Elastic MapReduce(EMR)라는 서비스를 제공, 직접 클러스터를 구성하는 것보다 쉬움
* 하둡에는 보통 지연 속도가 있어서 실시간 분석에는 적합하지 않음.
    * 실시간 분석에는 스파크(spark)가 인기

## code

In [2]:
# 단어수 세기
from typing import List, Iterator, Tuple, Iterable
from collections import Counter, defaultdict

def tokenize(document: str) -> List[str] :
    """공백 문자를 기준으로 나눈다."""
    return document.split()

def word_count_old(documents: List[str]) :
    """맵리듀스를 사용하지 않고 단어 수 세기"""
    return Counter(word for document in documents for word in tokenize(document))


# 각 단어를 만날 때 마다 한 번 등장했다는 것을 표시 
def wc_mapper(document: str) -> Iterator[Tuple[str, int]] :
    """문서의 각 단어마다 (단어,1)을 내보낸다."""
    for word in tokenize(document) :
        yield (word, 1)
        
def wc_reducer(word: str, counts: Iterable[int]) -> Iterator[Tuple[str,int]] :
    """단어의 모든 빈도 수를 더한다"""
    yield (word, sum(counts))

def word_count(documents: List[str]) -> List[Tuple[str, int]] :
    """맵리듀스를 사용해서 입력 문서의 단어 빈도 수를 세어 준다."""
    collector = defaultdict(list) # 쌍으로 묶인 값을 저장할 공간
    
    for document in documents:
        for word, count in wc_mapper(document) :
            collector[word].append(count)
    
    return [output 
            for word, count in collector.items()
            for output in wc_reducer(word, counts)]


[('data', 2), ('science', 2), ('big', 1), ()]

In [11]:
# 일반화를 위한 타입 어노테이션 
from typing import Callable, Iterable, Any, Tuple

# 키-값 쌍은 그저 값이 두 개인 튜플
KV = Tuple[Any, Any]

# Mapper는 키-값 쌍의 이터러블을 반환하는 함수
Mapper = Callable[..., Iterable[KV]]

# Reducer는 키와 값의 이터러블을 받고 키-값 쌍을 반환
Reducer = Callable[[Any, Iterable], KV]

# 일반화된 map_reuce함수
def map_reduce(inputs: Iterable, mapper: Mapper, reducer : Reducer) -> List[KV] :
    """mapper와 reducer를 사용해서 inputs에 맵리듀스를 적용"""
    collector = defaultdict(list)
    
    for input in inputs :
        for key, value in mapper(input) :
            collector[key].append(value)
    
    return [output 
            for key, values in collector.items()
            for output in reducer(key,values)]


In [15]:
documents = ["data science", "big data", "science fiction"]

# 단어 세기
word_counts = map_reduce(documents, wc_mapper, wc_reducer)

# wc_reducer는 각 키에 해당하는 값들을 단순히 더함 - 일반화
def values_reducer(values_fn : Callable) -> Reducer :
    """values_fn을 값에 적용하는 reducer 반환"""
    def reduce(key, values: Iterable) -> KV :
        return (key, values_fn(values))
    
    return reduce


In [16]:
word_counts

[('data', 2), ('science', 2), ('big', 1), ('fiction', 1)]

In [17]:
sum_reducer = values_reducer(sum)
max_reducer = values_reducer(max)
min_reducer = values_reducer(min)
count_distinct_reducer = values_reducer(lambda values: len(set(values)))

print(sum_reducer("key", [1,2,3,3]))
print(min_reducer("key", [1,2,3,3]))
print(max_reducer("key", [1,2,3,3]))
print(count_distinct_reducer("key", [1,2,3,3]))

('key', 9)
('key', 1)
('key', 3)
('key', 3)


In [19]:
import datetime

status_updates = [
    {"id" : 2,
     "username" : "joelgrus",
     "text" : "Should I write a second edition of my data science book?",
     "created_at" : datetime.datetime(2018, 2, 21, 11, 47, 0),
     "liked_by" : ["data_guy", "data_gal", "mike"]},
    # ...
]


In [20]:
# data science라는 문구가 포함된 글이 발견될 때 마다 1을 출력, 최종적으로 모든 숫자들을 sum으로 더함
def data_science_day_mapper(status_update: dict) -> Iterable :
    """status_update에 "data science"가 포함되어 있으면 (day_of_week, 1)을 내보낸다."""
    if "data science" in status_update["text"].lower() :
        day_of_week = status_update["created_at"].weekday()
        yield (day_of_week, 1)
    
data_science_day = map_reduce(status_updates, data_science_day_mapper, sum_reducer)

In [21]:
data_science_day

[2, 1]

In [23]:
# 각 사용자가 어떤 단어를 가장 많이 사용했는지 분석
def words_per_user_mapper(status_update: dict) :
    user = status_update["username"]
    for word in tokenize(status_update["text"]) :
        yield (user, (word, 1))

def most_popular_word_reducer(user: str, words_and_counts: Iterable[KV]) :
    """(단어, 빈도 수)쌍으로 구성된 배열에서 총 빈도 수가 가장 높은 단어를 반환"""
    word_counts = Counter()
    for word, count in words_and_counts :
        word_counts[word] += count
    
    word, count = word_counts.most_common(1)[0]
    
    yield (user, (word, count))
    
user_words = map_reduce(status_updates, words_per_user_mapper, most_popular_word_reducer)


In [24]:
def liker_mapper(status_update: dict) :
    user = status_update["username"]
    for liker in status_update["liked_by"] :
        yield (user, liker)

distinct_likers_per_user = map_reduce(status_updates, liker_mapper, count_distinct_reducer)

In [25]:
user_words

[('joelgrus', ('Should', 1))]

In [26]:
distinct_likers_per_user

['joelgrus', 3]

In [35]:
# 행렬 곱셈 예시

# C[i][j] = sum(A[i][x]*B[x][j] for x in range(m))

# sparse한 행렬 : 0이 아닌 값을 담고 있는 위치만 저장하면 효율적 

from typing import NamedTuple

class Entry(NamedTuple) :
    name : str
    i : int
    j : int
    value : float


In [36]:
def matrix_multiply_mapper(num_rows_a: int, num_cols_b : int) -> Mapper :
    # C[x][y] = A[x][0]*B[0][y] +...+A[x][m]*B[m][y]
    # 그러므로 A[i][j]는 모든 C[i][y]에 B[i][y]의 가중치로 들어감 
    # B[i][j]는 모든 C[x][j]에 A[x][i]의 가중치로 들어감
    
    def mapper(entry: Entry) :
        if entry.name == "A" :
            for y in range(num_cols_b) :
                key = (entry.i, y)             # C의 어떤 원소
                value = (entry.j, entry.value) # 합의 어떤 entry
                yield (key, value)
        else : 
            for x in range(num_rows_a) :
                key = (x, entry.j)             # C의 어떤 원소
                value = (entry.i, entry.value) # 합의 어떤 엔트리 
                yield (key,value)
    return mapper 


def matrix_multiply_reducer(key: Tuple[int, int], indexed_values: Iterable[Tuple[int, int]]) :
    results_by_index = defaultdict(list)
    
    for index, value in indexed_values :
        results_by_index[index].append(value)
    
    # 값이 두 개인 위치의 값을 곱하고 
    # (A에서 하나, B에서 하나) 이를 더한다.
    sumproduct = sum(values[0] * values[1]
                     for values in results_by_index.values()
                     if len(values) == 2)
    
    if sumproduct != 0.0 :
        yield (key, sumproduct)


In [37]:
A = [[3, 2, 0],
     [0, 0, 0]]

B = [[4, -1, 0],
     [10, 0, 0],
     [0, 0, 0]]

entries = [Entry("A", 0, 0, 3), Entry("A", 0, 1, 2), Entry("B", 0, 0, 4), Entry("B", 0, 1, -1), Entry("B", 1, 0, 10)]

mapper = matrix_multiply_mapper(num_rows_a = 2, num_cols_b = 3)
reducer = matrix_multiply_reducer


# 곱은 [[32, -3, 0],[0, 0, 0]]
# 그러므로 두 개의 항목이 있어야 한다.

print(set(map_reduce(entries, mapper, reducer)))


{((0, 0), 32), ((0, 1), -3)}
